In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
from more_itertools import sliced

In [ ]:
df = pd.read_csv('GoodReads_100k_books.csv').dropna()

In [44]:
regex = re.compile('[Ä®™ÙŠØ©Ð§‡Œ¯ƒŸ]')

In [36]:
def regex_filter(val):
    if val != None:
        mo = re.search(regex, str(val))
        if mo != None:
            return True
        else:
            return False
    else:
        return False

In [ ]:
to_clean = df[df['desc'].apply(regex_filter)]
to_clean["new_desc"] = ""
to_clean.to_csv('to_clean.csv', encoding='utf-8', index=False)

In [ ]:
to_clean = pd.read_csv('to_clean.csv')

In [ ]:
counter = 0 
def getNewDesc(url):
    global counter
    counter = counter + 1
    try:
        ourUrl = requests.get(str(url))
        soup = BeautifulSoup(ourUrl.text, 'html.parser')
        description = soup.findAll("div", {"id": "description"})

        if len(description) == 1:
            desc = description[0]
        else: 
            desc = description[1]

        spans = []
        for span in desc.select('span'):
            s = re.sub(r'<.*?>', '', str(span))
            spans.append(s)

        desc = ""
        if len(spans) == 1:
            desc = spans[0]
        elif len(spans) > 1:
            desc = spans[1]

        desc = " ".join(desc.split())
        to_clean.loc[to_clean.link == url, 'new_desc'] = desc

    except Exception as e:
        print(str(e) + " -> " + url)

    if counter % 50 == 0:
        to_clean.to_csv('to_clean.csv', encoding='utf-8', index=False)
        print(str(counter) + " -> escrevi")
    else:
        print(counter)
    return True

In [ ]:
to_clean_aux = to_clean.loc[to_clean.new_desc.isna()]
to_clean_aux

In [ ]:
to_clean_aux['link'].apply(getNewDesc)
to_clean.to_csv('to_clean.csv', encoding='utf-8', index=False)

In [51]:
clean = pd.read_csv('to_clean.csv')
regex = re.compile('[ÙŠØÐ§‡Œœ¼Å¢¯ƒŸ]')
clean = clean[clean["new_desc"].apply(lambda x: regex_filter(x) == False)]
clean.to_csv('cleanfile_desc.csv', encoding='utf-8', index=False)